In [1]:
import dotenv

dotenv.load_dotenv()

True

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm=ChatGoogleGenerativeAI(model="gemini-pro")

# llm = ChatVertexAI(model="gemini-pro")

c:\Users\moham\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [22]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("insea-actuariat.pdf")
docs = loader.load()

In [23]:
docs

[Document(page_content="COMPETENCES A ACQUERIR\n     Les ingénieurs diplômés de la filière Actuariat-Finance\nauront les compétences nécessaires pour occuper rapidement\ndes responsabilités de cadres supérieures dans le secteur des\nbanques et assurances. Les lauréats de la filière Actuariat-\nFinance seront capables de dominer les aspects technique et\nmanagérial qui les qualifient à s'insérer dans plusieurs\nsecteurs. La nature même de la formation en statistique de\nl’INSEA permet aux diplômés de la filière Actuariat-Finance\nd’être en premier des statisticiens capables de développer une\nexpertise scientifique de très haut niveau et d’appliquer leurs\ncompétences mathématiques, informatiques et statistiques\npour développer des modèles dans une grande variété de\ncontextes. \nDÉBOUCHÉS DE LA FORMATIONFILIÈRE \nACTUARIAT-FINANCE\nOBJECTIFS DE LA FORMATION \nFINANCIAL FREEDOM\nFOR X GENERATION          Le programme de la filière a connu une évolution importante d’une\naccréditation à

In [26]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
splits = text_splitter.split_documents(docs)

In [27]:
splits

[Document(page_content="COMPETENCES A ACQUERIR\n     Les ingénieurs diplômés de la filière Actuariat-Finance\nauront les compétences nécessaires pour occuper rapidement\ndes responsabilités de cadres supérieures dans le secteur des\nbanques et assurances. Les lauréats de la filière Actuariat-\nFinance seront capables de dominer les aspects technique et\nmanagérial qui les qualifient à s'insérer dans plusieurs\nsecteurs. La nature même de la formation en statistique de\nl’INSEA permet aux diplômés de la filière Actuariat-Finance\nd’être en premier des statisticiens capables de développer une\nexpertise scientifique de très haut niveau et d’appliquer leurs\ncompétences mathématiques, informatiques et statistiques\npour développer des modèles dans une grande variété de\ncontextes. \nDÉBOUCHÉS DE LA FORMATIONFILIÈRE \nACTUARIAT-FINANCE\nOBJECTIFS DE LA FORMATION \nFINANCIAL FREEDOM\nFOR X GENERATION          Le programme de la filière a connu une évolution importante d’une", metadata={'sou

In [33]:
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings


vectorstore = FAISS.from_documents(splits, GoogleGenerativeAIEmbeddings(model="models/embedding-001"))

In [34]:

docs_search = vectorstore.similarity_search("quelles sont les COMPETENCES des diplomés de la filiere actuaiat-finance de l'insea", k=5)
for doc in docs_search:
    print(doc.page_content[:200])
    print("\n")

COMPETENCES A ACQUERIR
     Les ingénieurs diplômés de la filière Actuariat-Finance
auront les compétences nécessaires pour occuper rapidement
des responsabilités de cadres supérieures dans le secteur


accréditation à une autre dans l’objectif de s’adapter aux besoins du marché et
aux changements de la réglementation en Finance et Assurance. Il s’agit d’une
formation solide dans les deux domaines le


Ingénieur recherche et développement « quants » 
Financial and Actuarial Data scientist
Traders 
Ingénieurs financiers 
Gestionnaire de portefeuille      Les lauréats de la filière Actuariat-Finance p


comptables, fiscaux et commerciaux dans lesquels se situe l’intervention de
l’ingénieur actuaire financier. Le cursus de la filière Actuariat-Finance se
caractérise par la richesse des cours avancés e


SEMESTRE 1 
PROBABILITES 
STATISTIQUE DESCRIPTIVE 
ALGORITHMIQUE & PROGRAMMATION 
GESTION DES BASES DE DONNEES 
METHODES NUMERIQUES 
MATHEMATIQUES FINANCIERES 
HISTOIRE ET PRINCIPES DE L’E

In [36]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()

prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [38]:
rag_chain.invoke("quelles sont les COMPETENCES des diplomés de la filiere actuaiat-finance de l'insea")

"Les diplômés de la filière actuariat-finance de l'INSEA acquièrent les compétences nécessaires pour occuper des postes de cadres supérieurs dans les secteurs bancaires et des assurances. Ils maîtrisent les aspects techniques et managériaux, et sont capables de développer des modèles dans divers contextes grâce à leurs compétences mathématiques, informatiques et statistiques. La formation leur permet également de dominer les aspects juridiques, comptables, fiscaux et commerciaux liés à leur domaine."